In [1]:
# Clear any logs from previous runs

In [2]:
import numpy as np
import pandas as pd
import time, os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.utils import to_categorical
from tensorboard.plugins.hparams import api as hp

from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Model

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split as split

In [3]:
np.random.seed(0)

tf.config.optimizer.set_jit(True)
scaler = StandardScaler()

### Conjunto de datos: SDSS DR17

In [4]:
url = "https://raw.githubusercontent.com/igomezv/nnogada/main/data/star_classification.csv"
data = pd.read_csv(url)
cols = ['alpha','delta','u','g','r','i','z','class']
data = data[cols]
data.head()

,alpha,delta,u,g,r,i,z,class
0,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,GALAXY
1,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,GALAXY
2,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,GALAXY
3,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,GALAXY
4,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,GALAXY


In [5]:
data["class"]=[0 if i == "GALAXY" else 1 if i == "STAR" else 2 for i in data["class"]]
print(data.head())
data = data.to_numpy()

        alpha      delta         u         g         r         i         z  \
0  135.689107  32.494632  23.87882  22.27530  20.39501  19.16573  18.79371   
1  144.826101  31.274185  24.77759  22.83188  22.58444  21.16812  21.61427   
2  142.188790  35.582444  25.26307  22.66389  20.60976  19.34857  18.94827   
3  338.741038  -0.402828  22.13682  23.77656  21.61162  20.50454  19.25010   
4  345.282593  21.183866  19.43718  17.58028  16.49747  15.97711  15.54461   

   class  
0      0  
1      0  
2      0  
3      0  
4      0  


In [6]:
# Divide data into X and Y and implement hot_ones in Y
def prepare_dataset(data):
    X, Y = np.empty((0)), np.empty((0))
    X = data[:,0:7]
    Y = data[:,7]
    Y = to_categorical(Y, num_classes=3)
    return X, Y

In [7]:
# Split dataset into train, validation and test sets
X,Y = prepare_dataset(data)

# Defines ratios, w.r.t. whole dataset.
ratio_train = 0.8
ratio_val = 0.1
ratio_test = 0.1

# Produces test split.
x_, X_test, y_, Y_test = split(X, Y, test_size = ratio_test, random_state=0)

# Adjusts val ratio, w.r.t. remaining dataset.
ratio_remaining = 1 - ratio_test
ratio_val_adjusted = ratio_val / ratio_remaining

# Produces train and val splits.
X_train, X_val, Y_train, Y_val = split(x_, y_, test_size=ratio_val_adjusted, random_state=0)

# Normalize and scale the input sets.
scaler.fit(X)
X_train = scaler.transform(X_train)
X_test  = scaler.transform(X_test)
X_val   = scaler.transform(X_val)

lenx, input_shape = np.shape(X)

### Hiperparámetros del modelo

In [8]:
HP_LAYERS =    hp.HParam('layers', hp.Discrete([1, 2, 3, 4]))
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([50, 100, 150, 200]))
HP_LEARNING  = hp.HParam('learning_rate', hp.Discrete([1e-5,1e-4,1e-3,0.01]))
# HP_BATCHSIZE = hp.HParam('batch_size', hp.Discrete([64,128,256,512]))

callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', mode='max',
                                   min_delta=0,
                                   patience=6,
                                   restore_best_weights=True)]
batch_size = 128
epochs = 50

In [9]:
# METRIC_ACCURACY = 'accuracy'
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
# with tf.summary.FileWriter('logs/hparam_tuning', sess.graph):
#     init = tf.initialize_all_variables()
#     sess.run(init)
    hp.hparams_config(
        hparams=[HP_LAYERS,
                 HP_NUM_UNITS,
                 HP_LEARNING],
        metrics=[hp.Metric('loss', display_name="Accuracy")])

In [10]:
def train_test_model(hparams):    
    
    # Train LSTM model and predict on validation set
    model = tf.keras.Sequential()
    model.add(Input(shape=(int(X_train.shape[1]),)))
    model.add(Dense(hparams[HP_NUM_UNITS], input_shape=(int(X_train.shape[1]),)))
    
    for i in range(hparams[HP_LAYERS]):        
        model.add(Dense(hparams[HP_NUM_UNITS], activation='relu'))
    model.add(Dense(3, activation=tf.nn.softmax))
     
    optimizer = tf.keras.optimizers.Adam(learning_rate=hparams[HP_LEARNING], beta_1=0.9, beta_2=0.999, epsilon=1e-3)
    model.compile(
            optimizer=optimizer,
            loss=CategoricalCrossentropy(),
            metrics=["categorical_accuracy"])
    
    # Run with 1 epoch to speed things up for demo purposes

    model.fit(X_train, Y_train, epochs=epochs, validation_data=(X_test, Y_test),
              callbacks=callbacks, batch_size=batch_size, shuffle=True, verbose=0)

    _, loss = model.evaluate(X_test, Y_test)
    
    return loss

In [11]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        loss = train_test_model(hparams)
        tf.summary.scalar("loss", loss, step=1)
        return loss

In [12]:
session_num = 0
datos = []

for deep_layers in HP_LAYERS.domain.values:
    for num_units in HP_NUM_UNITS.domain.values:
        for learning_rate in HP_LEARNING.domain.values:
#             for batch_size in HP_BATCHSIZE.domain.values:
            t = time.time()
            hparams = {

                HP_LAYERS: deep_layers,
                HP_NUM_UNITS: num_units,
                HP_LEARNING: learning_rate,
#                     HP_BATCHSIZE: batch_size
            }
            run_name = "run-%d" % session_num
            print('\n--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            score = run('logs/hparam_tuning/' + run_name, hparams)
            t = time.time()-t
            session_num += 1
            print("Accuracy:", score, "Tiempo transcurrido:", t)
            
            datos.append([deep_layers, num_units, learning_rate, score, t])

print(session_num)


--- Starting trial: run-0
{'layers': 1, 'num_units': 50, 'learning_rate': 1e-05}
313/313 [==============================] - 0s 845us/step - loss: 0.9230 - categorical_accuracy: 0.6327
Accuracy: 0.6327000260353088 Tiempo transcurrido: 7.428928852081299

--- Starting trial: run-1
{'layers': 1, 'num_units': 50, 'learning_rate': 0.0001}
313/313 [==============================] - 0s 919us/step - loss: 0.5297 - categorical_accuracy: 0.8097
Accuracy: 0.8097000122070312 Tiempo transcurrido: 35.90795302391052

--- Starting trial: run-2
{'layers': 1, 'num_units': 50, 'learning_rate': 0.001}
313/313 [==============================] - 0s 882us/step - loss: 0.3996 - categorical_accuracy: 0.8632
Accuracy: 0.8632000088691711 Tiempo transcurrido: 35.70741653442383

--- Starting trial: run-3
{'layers': 1, 'num_units': 50, 'learning_rate': 0.01}
313/313 [==============================] - 0s 833us/step - loss: 0.4241 - categorical_accuracy: 0.8544
Accuracy: 0.8543999791145325 Tiempo transcurrido: 12.930

313/313 [==============================] - 0s 636us/step - loss: 0.5713 - categorical_accuracy: 0.7878
Accuracy: 0.7878000140190125 Tiempo transcurrido: 36.67670202255249

--- Starting trial: run-33
{'layers': 3, 'num_units': 50, 'learning_rate': 0.0001}
313/313 [==============================] - 0s 610us/step - loss: 0.4592 - categorical_accuracy: 0.8393
Accuracy: 0.8392999768257141 Tiempo transcurrido: 29.101157426834106

--- Starting trial: run-34
{'layers': 3, 'num_units': 50, 'learning_rate': 0.001}
313/313 [==============================] - 0s 607us/step - loss: 0.3495 - categorical_accuracy: 0.8787
Accuracy: 0.8787000179290771 Tiempo transcurrido: 24.120343923568726

--- Starting trial: run-35
{'layers': 3, 'num_units': 50, 'learning_rate': 0.01}
313/313 [==============================] - 0s 649us/step - loss: 0.3374 - categorical_accuracy: 0.8807
Accuracy: 0.8806999921798706 Tiempo transcurrido: 24.824384689331055

--- Starting trial: run-36
{'layers': 3, 'num_units': 100, 'lea

### Guardar datos

In [13]:
filename = "historial_sdss_tunning.txt"
df = pd.DataFrame(datos, columns = ["Deep size", "Num units", "Learning rate", "Accuracy", "Tiempo de ejecución"])

df.sort_values(by=["Accuracy", "Tiempo de ejecución"], ascending=[0,0], ignore_index=True, inplace=True)

df.to_csv(filename, header=True, index=False, sep='\t', mode='w') # a=append, w=overwrite

In [14]:
df

,Deep size,Num units,Learning rate,Accuracy,Tiempo de ejecución
0,2,150,0.00100,0.8844,45.322384
1,4,100,0.00100,0.8840,34.670704
2,3,200,0.00100,0.8839,36.891528
3,3,100,0.00100,0.8824,29.720098
4,3,150,0.00100,0.8819,29.672873
...,...,...,...,...,...
59,1,150,0.00001,0.7813,44.285113
60,2,50,0.00001,0.7799,37.557738
61,1,200,0.00001,0.7794,42.390247
62,1,100,0.00001,0.7762,39.456221


In [15]:
np.sum(df[["Tiempo de ejecución"]])/60

Tiempo de ejecución    39.449335
dtype: float64

In [16]:
# rm -rf /tmp/tb_logs/

### Now in terminal:
`python3 -m tensorboard.main --logdir='/home/isidro/Documents/github/neurapprox/logs/hparam_tuning'`